In [ ]:
import os

from src.load_data import load_matches
from src.offball_wrappers import obr_per_subtype_per90min, obr_per_subtype_per_phase
#from src.plots import plot_total_and_untargeted_per90

Total off-ball runs by subtype + untargeted share
% of run types by phase of play
Two radar plots: teams and players

and then sth to move to midfielders
and then the DEEP ANALYSIS and metrics creation

In [2]:
# load matches
matches_json_path = os.path.join(os.getcwd(), "src/data/matches.json")
all_tracking, dynamic_events_all, all_metadata = load_matches(matches_json_path)

c:\Users\g4a4b\OneDrive - University of Twente\SportsAnalyticsProject\analytics_cup_research\src\load_data.py:54: DtypeWarning: Columns (75,77,184,264) have mixed types. Specify dtype option on import or set low_memory=False.
  de_match = pd.read_csv(url)
c:\Users\g4a4b\OneDrive - University of Twente\SportsAnalyticsProject\analytics_cup_research\src\load_data.py:54: DtypeWarning: Columns (276) have mixed types. Specify dtype option on import or set low_memory=False.
  de_match = pd.read_csv(url)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

subtype_names_map = {
    "behind": "Behind",
    "coming_short": "Coming Short",
    "cross_receiver": "Cross Receiver",
    "dropping_off": "Dropping Off",
    "overlap": "Overlap",
    "pulling_half_space": "Pulling Half-Space",
    "pulling_wide": "Pulling Wide",
    "run_ahead_of_the_ball": "Run Ahead",
    "support": "Support",
    "underlap": "Underlap"
}

def plot_total_and_untargeted_per90(agg_df: pd.DataFrame, title: str = "Off-Ball Events per 90 min"):
    """
    Plots horizontal bars of total off-ball events per 90 min per subtype,
    overlaying the proportion of untargeted events with green bars with black edges.

    Args:
        agg_df (pd.DataFrame): DataFrame returned by `ofb_per_subtype_per90min`.
            Must contain columns:
                - "subtype"
                - "total_per90min"
                - "untargeted_per90min"
        title (str): Plot title.
    """

    # Compute proportion of untargeted per total
    agg_df = agg_df.copy()
    agg_df["untargeted_proportion"] = agg_df["untargeted_per90min"] / agg_df["total_per90min"]
    agg_df["subtype_display"] = agg_df["event_subtype"].map(subtype_names_map)

    # Sort by total events for better visualization
    agg_df = agg_df.sort_values("total_per90min", ascending=True)

    plt.figure(figsize=(10, 6))
    ax = plt.gca()

    # Plot total events per 90 min as base bars (light gray)
    ax.barh(
        agg_df["subtype_display"],
        agg_df["total_per90min"],
        color="#d3d3d3",
        edgecolor="black",
        linewidth=1.0,
        label="Total per 90 min"
    )

    # Overlay untargeted proportion as green bars
    ax.barh(
        agg_df["subtype_display"],
        agg_df["total_per90min"] * agg_df["untargeted_proportion"],
        color="#00ff1e",
        edgecolor="black",
        linewidth=1.5,
        label="% Untargeted"
    )

    # Aesthetics: remove top/right spines, keep left/bottom
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_visible(True)

    # Remove grid
    plt.grid(False)

    # Y-axis labels
    plt.yticks(fontsize=12, fontweight='normal', fontname='Arial')

    # X-axis label
    plt.xlabel("Events per 90 min", fontsize=12, fontweight='bold')

    # Title
    plt.title(title, fontsize=14, fontweight='bold')

    # Legend
    ax.legend(loc="upper right")

    plt.tight_layout()
    plt.show()


In [12]:
# compute off-ball event rates per subtype per 90 minutes
ofb_per_subtype_per90min_df = obr_per_subtype_per90min(all_tracking, dynamic_events_all)
print("Off-ball events per subtype per 90 minutes:")
print(ofb_per_subtype_per90min_df)

# compute off-ball event rates per subtype per phase
ofb_per_subtype_per_phase_df = obr_per_subtype_per_phase(dynamic_events_all)
print("Off-ball events per subtype per phase:")
print(ofb_per_subtype_per_phase_df)


# plot total and untargeted off-ball events per 90 minutes
plot_total_and_untargeted_per90(ofb_per_subtype_per90min_df)

Total played: 97.4 minutes
Total played: 99.8 minutes
Total played: 94.7 minutes
Total played: 98.8 minutes
Total played: 98.7 minutes
Total played: 96.0 minutes
Total played: 98.7 minutes
Total played: 102.1 minutes
Total played: 100.8 minutes
Total played: 98.4 minutes
Nan values in 'targeted' column: 0
Off-ball events per subtype per 90 minutes:
           event_subtype  total targeted untargeted  total_per90min  \
0                 behind    363      155        208       33.148611   
1           coming_short    701      244        457       64.014259   
2         cross_receiver    423      130        293       38.627720   
3           dropping_off    631      245        386       57.621965   
4                overlap    153       60         93       13.971728   
5     pulling_half_space    148       35        113       13.515136   
6           pulling_wide    344      154        190       31.413559   
7  run_ahead_of_the_ball   1378      475        903      125.836875   
8         

KeyError: 'subtype'